In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.misc 
import scipy.io
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [9]:
# 1. – Load costum dataset and build training and test data tensor

# 1.1.  Load cat/dog image data generated by generate_dataset.ipynb


cwd = os.getcwd()
loadpath = cwd + "/data/data4vgg.npz"
l = np.load(loadpath)

# See what's in here
l.files

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]

print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("%d classes" % (nclass))

69 train images loaded
18 test images loaded
37632 dimensional input
2 classes


In [10]:
# 1.2. Make a training_tensor and test_tensor each of which has (n, 112, 112, 3) dimension. np.ndarray, np.reshape

training_tensor = np.ndarray((ntrain, 112, 112, 3))

# for each training example, reshape it and update training_tensor
# we have 69 training images, training tensor dimension will be 69, 112, 112, 3
for index in range(ntrain):
    img = trainimg[index, :]
    img = np.reshape(img, [112, 112, 3])
    training_tensor[index, :, :, :] = img 
    
test_tensor = np.ndarray((ntest, 112, 112, 3))

# do the same thing for each test example
# we have 18 test images, test tensor dimension will be 18, 112, 112, 3
for index in range(ntest):
    img = testimg[index, :]
    img = np.reshape(img, [112, 112, 3])
    test_tensor[index, :, :, :] = img

In [11]:
# 2. – Load pre-trained VGG model as same way in vgg.ipynb

# code from vgg.ipynb
def net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current

    assert len(net) == len(layers)
    return net, mean_pixel

def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel

In [12]:
# 3. Define VGG features as the output of relu5_4

def imread(path):
    return scipy.misc.imread(path).astype(np.float)
def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    scipy.misc.imsave(path, img)
        
cwd  = os.getcwd()
VGG_PATH = cwd + "/data/imagenet-vgg-verydeep-19.mat"
with tf.Graph().as_default(), tf.Session() as sess:
    # as given in assignment5.pdf file by default
    image_placeholder = tf.placeholder(tf.float32, shape=(None, 112, 112, 3))
    net, mean_pixel = net(VGG_PATH, image_placeholder)
    train_features = net['relu5_4'].eval(feed_dict={image_placeholder: training_tensor})
    test_features = net['relu5_4'].eval(feed_dict={image_placeholder: test_tensor})
    

In [13]:
# 4. – Construct your fully connected layers on top of the VGG features

# 4.1. • You would need to vectorize VGG features to be input of the fc layer
# 4.2. • Try to build multiple layers between 7x7x512 dimensional input to 2-dim output

training_tensor_vect = np.ndarray((ntrain, 7 * 7 * 512))
test_tensor_vect = np.ndarray((ntest, 7 * 7 * 512))

# reshape each training feature obtained from relu5_4 layer and update vectorized tensor
for index in range(ntrain):
    feature = train_features[index, :, :, :]
    feature_reshaped = np.reshape(feature, (1, -1))
    training_tensor_vect[index, :] = feature_reshaped

# reshape each test feature obtained from relu5_4 layer and update vectorized tensor
for index in range(ntest):
    feature = test_features[index, :, :, :]
    feature_reshaped = np.reshape(feature, (1, -1))
    test_tensor_vect[index, :] = feature_reshaped

In [14]:
# 5. – Training fc layers using your training images. (Fine-tuning)

# Parameters
learning_rate   = 0.001
training_epochs = 50
batch_size      = 100
display_step    = 10

# Fully Connected Layer
n_input  = dim
n_output  = 2 # cat or dog

# tf Graph input
x = tf.placeholder(tf.float32, [None, 7 * 7 * 512])
y = tf.placeholder(tf.float32, [None, n_output])

# keep probability for dropout
keep_prob = tf.placeholder(tf.float32)

# define weights and biases
weights  = {
    'wf1': tf.Variable(tf.random_normal([7 * 7 * 512, 1024], stddev=0.1)),
    'wf2': tf.Variable(tf.random_normal([1024, n_output], stddev=0.1))
}
biases   = {
    'bf1': tf.Variable(tf.random_normal([1024], stddev = 0.1)),
    'bf2': tf.Variable(tf.random_normal([n_output], stddev = 0.1))
}

# Vectorize
dense = tf.reshape(x, [-1, weights['wf1'].get_shape().as_list()[0]])
    
# layer before output layer
before_out = tf.nn.relu(tf.add(tf.matmul(dense, weights['wf1']), biases['bf1']))
# apply dropout
before_out_drop = tf.nn.dropout(before_out, keep_prob)
    
# output layer
out = tf.add(tf.matmul(before_out_drop, weights['wf2']), biases['bf2'])

# Saver 
save_step = 10;
saver = tf.train.Saver(max_to_keep=training_epochs) 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
corr = tf.equal(tf.argmax(out,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(corr, tf.float32)) # Accuracy

# init all variables
init = tf.initialize_all_variables()

In [16]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = training_tensor_vect[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keep_prob:1.})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob:1.}) / num_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keep_prob:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: test_tensor_vect, y: testlabel, keep_prob:1.})
        print (" Test accuracy: %.3f" % (test_acc))
            
    # Save Net
    if epoch % save_step == 0:
        saver.save(sess, "net/cnn_custom_dataset.ckpt-" + str(epoch))

print ("Optimization Finished!")

# 6. I obtained test accuracy of 0.899 which is bigger than 0.833. By playing with dropout we can improve accuracy more.

Epoch: 000/050 cost: 66.911224365
 Training accuracy: 0.550
 Test accuracy: 0.222
Epoch: 010/050 cost: 2.996534348
 Training accuracy: 0.830
 Test accuracy: 0.333
Epoch: 020/050 cost: 0.077286795
 Training accuracy: 0.980
 Test accuracy: 0.833
Epoch: 030/050 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 0.889
Epoch: 040/050 cost: 0.000000000
 Training accuracy: 1.000
 Test accuracy: 0.889
Optimization Finished!
